In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

data = pd.read_csv('/kaggle/input/movielens/ratings.csv')
print(data.head())

plt.figure(figsize = (8, 8))
data.rating.hist(grid=False)
plt.ylabel('Frequency')
plt.ylabel('Rating')
plt.title('Rating Distribution')
plt.show()

In [ ]:
# 第 1 部分
# 載入函式庫與資料集
from keras.layers import Input, Embedding, Flatten, Dot
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn import metrics

import numpy as np
import pandas as pd

np.random.seed(123456)
data = pd.read_csv('/kaggle/input/movielens/ratings.csv')

In [ ]:
data.head

In [ ]:
# 第 2 部分
# 資料清理
def get_data(data):

    # 刪除時間
    data.drop('timestamp', axis=1, inplace=True)
    
    # 抓出所有使用者跟電影索引
    users = data.userId.unique()
    movies = data.movieId.unique()
 
    # 找出新索引跟原始索引的對應關係
    moviemap={}
    for i in range(len(movies)):
        moviemap[movies[i]]=i
    usermap={}
    for i in range(len(users)):
        usermap[users[i]]=i
    
    # 更改成新索引成連續整數值
    data.movieId = data.movieId.apply(lambda x: moviemap[x])    
    data.userId = data.userId.apply(lambda x: usermap[x])    
        
    # 打亂資料
    data = data.sample(frac=1.0).reset_index(drop=True)
    
    # 建立訓練資料、測試資料集
    train, test = train_test_split(data, test_size=0.2)
    
    n_users = len(users)
    n_movies = len(movies)

    return train, test, n_users, n_movies

train, test, n_users, n_movies = get_data(data)

In [ ]:
print(train)
print(test)

In [ ]:
# 第 3 部分
# 建立神經網路結構

fts = 5

# 輸入層接收資料
# 嵌入層將資料轉換成5維矩陣
# 展平層將5維矩陣拉直成陣列
movie_in = Input(shape=[1], name="Movie")
mov_embed = Embedding(n_movies, 
                      fts, 
                      name="Movie_Embed")(movie_in)
flat_movie = Flatten(name="FlattenM")(mov_embed)

user_in = Input(shape=[1], name="User")
user_inuser_embed = Embedding(n_users, 
                              fts, 
                              name="User_Embed")(user_in)
flat_user = Flatten(name="FlattenU")(user_inuser_embed)

# 計算點積
prod = Dot(name="Mult", axes=1)([flat_movie, flat_user])

# 編譯模型
model = Model([user_in, movie_in], prod)
model.compile('adam', 'mean_squared_error')
model.summary()


In [ ]:
# 第 4 部分
# 訓練神經網路
model.fit([train.userId, train.movieId], 
          train.rating, 
          epochs=10, 
          verbose=1)

# 評估神經網路
print("MSE:", 
      metrics.mean_squared_error(test.rating, 
                                 model.predict([test.userId, 
                                                test.movieId])))
